In [ ]:
project_path = "/home/jupyter"
import os
import sys

sys.path.append(project_path)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from google.cloud import bigquery

from fintrans_toolbox.src import bq_utils as bq
from fintrans_toolbox.src import table_utils as t


client = bigquery.Client()


In [ ]:
#UK Visa Domestic Online Goods Total

UK_spending_by_mccgoods_yearly_D = '''
SELECT 
    CAST(SUBSTR(time_period_value, 1, 4) AS INT64) AS year,  -- Extract the first 4 characters (year) from time_period_value
    SUM(spend) AS mcc_goods_total_D  -- Sum the total spend for each year
FROM  
    `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
WHERE  
    mcc IN ( 
        'ANTIQUE REPRODUCTION STORES',
'ANTIQUE SHOPS', 
        'ARTIST/CRAFT SHOPS',
        'ART DEALERS & GALLERIES',
        'AUTOMATED FUEL DISPENSERS',
        'AUTOMOTIVE PARTS STORES', 
        'AUTOMOTIVE TIRE STORES', 
        'BAKERIES', 
        'BICYCLE SHOPS/SALE/SERVICE',
        'BOAT DEALERS',
        'BOOKS/PERIODICALS/NEWSPAPERS',
        'BOOK STORES', 
        'CAMERA & PHOTO SUPPLY STORES',
        'CAMPER TRAILER DEALER',
        'CANDY/NUT/CONFECTION STORES',
        'CAR & TRUCK DEALERS/NEW/USED', 
        'CAR & TRUCK DEALERS/USED ONLY', 
        'CATALOG MERCHANT',
        'CHILDREN/INFANTS WEAR STORES',
        'CLOTHING/RENT/COSTUME/UNIFO',
        'COMBINATION CATALOG & RETAIL',
        'COMPUTERS/PERIPHERALS/SOFTWARE',
        'COSMETIC STORES', 
        'DAIRY PRODUCT STORES',
        'DEPARTMENT STORES', 
        'DIRECT SELL/DOOR-TO-DOOR',
        'DISCOUNT STORES',
        'DRAPERY & UPHOLSTERY STORES',
        'DUTY FREE STORES',
        'ELEC RAZOR STORES/SALE/SERV',
        'ELECTRONICS STORES', 
        'FABRIC STORES',
        'FAMILY CLOTHING STORES',
        'FIREPLACES & ACCESSORIES',
        'FLOOR COVERING STORES', 
        'FLORIST SUPPLIES/NURSERY STOCK',
        'FLORISTS',
        'FREEZER/MEAT LOCKERS',
        'FUEL DEALERS',
        'FURNITURE/EQUIP STORES',
        'FURRIERS AND FUR SHOPS',
        'GLASS/PAINT/WALLPAPER STORES', 
        'GIFT, CARD, NOVELTY STORES', 
        'GLASSWARE/CRYSTAL STORES',
        'GROCERY STORES/SUPERMARKETS',
        'HARDWARE STORES',
        'HOBBY, TOY & GAME STORES',
        'HOME SUPPLY WAREHOUSE STORES',
        'HOUSEHOLD APPLIANCE STORES', 
        'JEWELRY STORES', 
        'LUGGAGE/LEATHER STORES',
        'LUMBER/BUILD. SUPPLY STORES', 
        'MEN/BOYS CLOTHING/ACC STORES',
        'MENS/WOMENS CLOTHING STORES',
        'MISC APPAREL/ACCESS STORES',
        'MISC AUTO DEALERS - DEFAULT',
        'MISC FOOD STORES - DEFAULT',
        'MISC GENERAL MERCHANDISE', 
        'MISC HOME FURNISHING SPECIALTY', 
        'MISC SPECIALTY RETAIL', 
        'MOBILE HOME DEALERS', 
        'MOTOR HOME DEALERS',
        'MOTOR VEHICLE SUPPLY/NEW PARTS', 
        'MOTORCYCLE DEALERS', 
        'MUSIC STORES/PIANOS', 
        'NEWS DEALERS/NEWSSTANDS',
        'NURSURIES, LAWN/GARDEN SUPPLY',
        'OFFICE/PHOTO EQUIPMENT',
        'ONLINE MARKETPLACES',
        'OTHER DIRECT MARKETERS',
        'OUTBOUND TELEMARKETING MERCHNT',
        'PAINT, VARNISHES & SUPPLIES',
        'PET STORES/FOOD & SUPPLY',
        'PETROLEUM/PETROLEUM PRODUCTS',
        'PLUMBING/HEATING EQUIPMENT',
        'POSTAGE STAMPS',
        'PRECIOUS STONES/METALS/JEWELRY',
        'RECORD STORES',
        'RELIGIOUS GOODS STORES',
        'ROOFING/SIDING/SHEET METAL'
        'SERVICE STATIONS',
        'SHOE STORES',
        'SNOWMOBILE DEALERS',
        'SPORTING GOODS STORES',
        'SPORTS/RIDING APPAREL STORES',
        'STAMP & COIN STORES',
        'STATIONERY STORES',
        'STATIONERY/OFFICE SUPPLIES',
        'SWIMMING POOLS/SALES/SERV',
        'TENT AND AWNING SHOPS',
        'TELECOMMUNICATION EQUIPMENT',
        'UNIFORMS & COMMERCIAL CLOTHING',
        'USED MERCHANDISE STORES',
        'WIG AND TOUPEE STORES',
        'WOMENS READY TO WEAR STORES',
        'WRECKING SALVAGE YARDS',
'VIDEO AMUSEMENT GAME SUPPLY'
    ) 
    AND time_period = 'Quarter' 
    AND merchant_channel = 'Online' 
    and cardholder_origin_country = 'All' 
and cardholder_origin = 'UNITED KINGDOM' 
and destination_country = 'UNITED KINGDOM'
    AND CAST(SUBSTR(time_period_value, 1, 4) AS INT64) BETWEEN 2019 AND 2024  -- Filter by year range
GROUP BY  
    year
ORDER BY  
    year'''

df_by_mccgoods_yearly_D = bq.read_bq_table_sql(client, UK_spending_by_mccgoods_yearly_D)
df_by_mccgoods_yearly_D

In [ ]:
#UK Visa Spend Domestic Total (Visa)
   year         spend
0  2019  5.155348e+11
1  2020  5.218997e+11
2  2021  5.509458e+11
3  2022  5.316535e+11
4  2023  5.108432e+11
5  2024  3.786755e+11

In [ ]:
#UK Visa Domestic Online Goods Ratio

import pandas as pd

# Data for mcc_goods_total
df_mcc_goods = pd.DataFrame({
    'year': [2019, 2020, 2021, 2022, 2023, 2024],
    'mcc_goods_total_D': [6.977591e+10, 1.028293e+11, 1.133436e+11, 9.791242e+10, 9.138777e+10, 9.443105e+10]
})

# Data for total_spending_visa
df_total_spending_visa = pd.DataFrame({
    'year': [2019, 2020, 2021, 2022, 2023, 2024],
    'total_spending_visa': [5.571616e+11, 5.584913e+11, 5.868134e+11, 5.665042e+11, 5.466160e+11, 4.075764e+11]
})

# Merging the two dataframes on the 'year' column
df_combined = pd.merge(df_mcc_goods, df_total_spending_visa, on='year')

# Calculating the Goods Online Abroad Ratio
df_combined['Goods_Online_Domestic_Ratio'] = (df_combined['mcc_goods_total_D'] / df_combined['total_spending_visa']) * 100

# Display the result
print(df_combined[['year', 'Goods_Online_Domestic_Ratio']])


In [ ]:
import pandas as pd
import plotly.express as px

# Data for UK Finance Domestic Online Total
domestic_online_total_data = {
    'year': [2019, 2020, 2021, 2022, 2023],
    'UK_Domestic_Online_Total': [3.0958e+11, 3.77357e+11, 4.02423e+11, 4.30713e+11, 4.62355e+11]
}

# Data for UK Finance Domestic Online Goods Total
domestic_online_goods_data = {
    'year': [2019, 2020, 2021, 2022, 2023],
    'UK_Finance_Goods_Online_Domestic_Total_Spending': [3.88e+10, 6.95e+10, 7.77e+10, 7.44e+10, 7.73e+10]
}

# Data for UK Finance Domestic Online Services Total
domestic_online_services_data = {
    'year': [2019, 2020, 2021, 2022, 2023],
    'UK_Finance_Services_Online_Domestic_Total': [2.71e+11, 3.08e+11, 3.25e+11, 3.56e+11, 3.85e+11]
}

# Create dataframes for each data set
df_domestic_online_total = pd.DataFrame(domestic_online_total_data)
df_domestic_online_goods = pd.DataFrame(domestic_online_goods_data)
df_domestic_online_services = pd.DataFrame(domestic_online_services_data)

# Merge dataframes on 'year'
df_domestic = pd.merge(df_domestic_online_total, df_domestic_online_goods, on='year')
df_domestic = pd.merge(df_domestic, df_domestic_online_services, on='year')

# Create the line chart
fig_line_domestic = px.line(df_domestic, 
                            x='year', 
                            y=['UK_Domestic_Online_Total', 'UK_Finance_Goods_Online_Domestic_Total_Spending', 'UK_Finance_Services_Online_Domestic_Total'], 
                            title='UK Finance Domestic Online Spending (Total, Goods, and Services) 2019-2023',
                            labels={'year': 'Year', 
                                    'UK_Domestic_Online_Total': 'Domestic Online Total (£)', 
                                    'UK_Finance_Goods_Online_Domestic_Total_Spending': 'Goods Online Total (£)', 
                                    'UK_Finance_Services_Online_Domestic_Total': 'Services Online Total (£)'},
                            markers=True)  # Add markers at each data point

# Customize the line chart appearance
fig_line_domestic.update_traces(line=dict(width=3), marker=dict(size=7))
fig_line_domestic.update_layout(xaxis=dict(tickmode='linear'), yaxis=dict(title='Value (£)'))

# Show the line chart
fig_line_domestic.show()